# WILDCAT Post-Fire Debris Flow Analysis - Complete Workflow
This master notebook automates the entire WILDCAT model workflow for post-fire debris flow analysis. 

## Workflow Overview:
1. **Data Preparation**: Extract MTBS bundles and organize fire data
2. **GEE Processing**: Authenticate, upload assets, and download DEMs
3. **Data Clipping**: Clip soil, EVT, and severity data to fire perimeters
4. **WILDCAT Execution**: Run debris flow assessments for all fires
5. **Visualization**: Generate probability raster maps

## 0. Setup and Configuration

In [ ]:
# Import required libraries
import os
import sys
import json
import time
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import our modules
from src.wildcat_automation import WildcatAutomation
from src.data_acquisition import MTBSExtractor, GEEDownloader
from src.preprocessing import DataValidator, FolderOrganizer
from src.visualization import RasterGenerator, BatchVisualizer

print("✅ All modules imported successfully!")

### Configuration

In [ ]:
# Configuration settings
CONFIG = {
    "paths": {
        "root_folder": "./data/fires",
        "shared_data": "./data/shared",
        "output_folder": "./outputs",
        "wildcat_projects": "./wildcat_projects"
    },
    "processing": {
        "years_to_process": ["2019", "2020", "2021"],  # Example years
        "max_fires_per_year": 5,  # Set to None for all fires
        "generate_rasters": True
    },
    "gee": {
        "project_id": "ee-yourproject",  # Update with your GEE project
        "asset_path": "projects/ee-yourproject/assets/fires"
    },
    "visualization": {
        "probability_thresholds": [16, 20, 24, 40],  # mm/hr
        "output_format": "GTiff",
        "color_scheme": "viridis"
    }
}

print("📋 Configuration loaded!")
print(f"   Years to process: {CONFIG['processing']['years_to_process']}")
print(f"   Output folder: {CONFIG['paths']['output_folder']}")

## 1. Data Preparation - Extract MTBS Bundles

In [ ]:
print("\n" + "="*60)
print("STEP 1: MTBS BUNDLE EXTRACTION")
print("="*60)

# Initialize extractor
extractor = MTBSExtractor(root_folder=CONFIG['paths']['root_folder'])

# Check for zip files
zip_count = extractor.count_zip_files(CONFIG['processing']['years_to_process'])
print(f"\n📦 Found {zip_count} MTBS bundles to extract")

if zip_count > 0:
    # Extract all bundles
    print("\n🔄 Extracting MTBS bundles...")
    extraction_results = extractor.extract_all_bundles(
        years=CONFIG['processing']['years_to_process']
    )
    
    print(f"\n✅ Extraction complete!")
    print(f"   Successful: {extraction_results['successful']}")
    print(f"   Failed: {extraction_results['failed']}")
else:
    print("\n⚠️  No zip files found. Assuming data is already extracted.")


## 2. Validate and Organize Fire Data

In [ ]:
print("\n" + "="*60)
print("STEP 2: DATA VALIDATION AND ORGANIZATION")
print("="*60)

# Initialize organizer
organizer = FolderOrganizer(root_folder=CONFIG['paths']['root_folder'])

# Get fire inventory
fire_inventory = organizer.get_fire_inventory(CONFIG['processing']['years_to_process'])
print(f"\n🔥 Found {len(fire_inventory)} fires to process")

# Display sample fires
print("\nSample fires:")
for i, (fire_name, fire_info) in enumerate(list(fire_inventory.items())[:5]):
    print(f"  {i+1}. {fire_name}")
    print(f"     - Perimeter: {'✅' if fire_info['has_perimeter'] else '❌'}")
    print(f"     - dNBR: {'✅' if fire_info['has_dnbr'] else '❌'}")


## 3. Google Earth Engine Processing

In [ ]:
print("\n" + "="*60)
print("STEP 3: GOOGLE EARTH ENGINE PROCESSING")
print("="*60)

# Initialize GEE downloader
gee_downloader = GEEDownloader(
    project_id=CONFIG['gee']['project_id'],
    asset_path=CONFIG['gee']['asset_path']
)

print("🌍 Authenticating with Google Earth Engine...")
try:
    import ee
    ee.Initialize(project=CONFIG['gee']['project_id'])
    print("✅ GEE authentication successful!")
except Exception as e:
    print(f"❌ GEE authentication failed: {e}")
    print("   Please run: earthengine authenticate")


### 3.1 Upload Fire Perimeters to GEE

In [ ]:
print("\n📤 Uploading fire perimeters to GEE...")

upload_results = gee_downloader.batch_upload_perimeters(
    fire_inventory,
    max_parallel=5
)

print(f"\n✅ Upload complete!")
print(f"   Successful: {upload_results['successful']}")
print(f"   Failed: {upload_results['failed']}")

### 3.2 Download DEMs for Each Fire

In [ ]:
print("\n📥 Downloading DEMs from GEE...")

dem_results = gee_downloader.batch_download_dems(
    fire_inventory,
    output_folder=CONFIG['paths']['root_folder'],
    max_parallel=5
)

print(f"\n✅ DEM download complete!")
print(f"   Successful: {dem_results['successful']}")
print(f"   Failed: {dem_results['failed']}")

## 4. Clip Shared Datasets

In [ ]:
print("\n" + "="*60)
print("STEP 4: CLIPPING SHARED DATASETS")
print("="*60)

# Paths to shared datasets
shared_paths = {
    'soil': os.path.join(CONFIG['paths']['shared_data'], 'soil', 'ussoils_18.shp'),
    'evt': os.path.join(CONFIG['paths']['shared_data'], 'evt', 'LC24_EVT_250.tif'),
    'severity_base': os.path.join(CONFIG['paths']['shared_data'], 'severity')
}

print("📍 Shared dataset paths:")
for dataset, path in shared_paths.items():
    exists = "✅" if os.path.exists(path) else "❌"
    print(f"   {dataset}: {exists} {path}")

## 5. Initialize WILDCAT Projects

In [ ]:
print("\n" + "="*60)
print("STEP 5: WILDCAT PROJECT INITIALIZATION")
print("="*60)

# Initialize WILDCAT automation
wildcat_auto = WildcatAutomation(
    root_folder=CONFIG['paths']['root_folder'],
    output_folder=CONFIG['paths']['output_folder'],
    shared_data_paths=shared_paths
)

# Initialize projects for all fires
print("\n🚀 Initializing WILDCAT projects...")
init_results = wildcat_auto.initialize_all_projects(
    fire_inventory,
    project_folder=CONFIG['paths']['wildcat_projects']
)

print(f"\n✅ Initialization complete!")
print(f"   Successful: {init_results['successful']}")
print(f"   Failed: {init_results['failed']}")

# Save initialization results
with open('initialized_projects.json', 'w') as f:
    json.dump(init_results['projects'], f, indent=2)

## 6. Run WILDCAT Assessments

In [ ]:
print("\n" + "="*60)
print("STEP 6: WILDCAT BATCH ASSESSMENT")
print("="*60)

# Load initialized projects
with open('initialized_projects.json', 'r') as f:
    initialized_projects = json.load(f)

print(f"📊 Processing {len(initialized_projects)} fires...")
print("   Each fire will be:")
print("   1. Preprocessed (clip all datasets)")
print("   2. Assessed (calculate debris flow probability)")
print("   3. Exported (create shapefiles)")

# Run batch assessment
assessment_results = wildcat_auto.run_batch_assessment(
    initialized_projects,
    skip_existing=True,
    memory_threshold_gb=2
)

# Summary
print(f"\n📊 ASSESSMENT SUMMARY:")
print(f"   Total fires: {len(assessment_results)}")
print(f"   ✅ Successful: {sum(1 for r in assessment_results.values() if r['success'])}")
print(f"   ❌ Failed: {sum(1 for r in assessment_results.values() if not r['success'])}")

# Save results
with open('assessment_results.json', 'w') as f:
    json.dump(assessment_results, f, indent=2, default=str)

## 7. Generate Probability Rasters

In [ ]:
print("\n" + "="*60)
print("STEP 7: GENERATING PROBABILITY RASTERS")
print("="*60)

# Initialize raster generator
raster_gen = RasterGenerator(
    output_folder=os.path.join(CONFIG['paths']['output_folder'], 'probability_rasters')
)

# Generate rasters for successful assessments
successful_fires = [
    (fire_name, project_info) 
    for fire_name, project_info in initialized_projects.items()
    if assessment_results.get(fire_name, {}).get('success', False)
]

print(f"\n🎨 Generating probability rasters for {len(successful_fires)} fires...")

raster_results = raster_gen.batch_generate_rasters(
    successful_fires,
    probability_columns=['P_16mmh', 'P_20mmh', 'P_24mmh', 'P_40mmh'],
    resolution=30  # meters
)

print(f"\n✅ Raster generation complete!")
print(f"   Generated: {raster_results['generated']} rasters")
print(f"   Failed: {raster_results['failed']}")

## 8. Create Summary Visualizations

In [ ]:
print("\n" + "="*60)
print("STEP 8: CREATING SUMMARY VISUALIZATIONS")
print("="*60)

# Initialize batch visualizer
visualizer = BatchVisualizer(
    output_folder=os.path.join(CONFIG['paths']['output_folder'], 'visualizations')
)

# Create composite maps
print("\n🗺️  Creating composite hazard maps...")

# 1. Overview map of all fires
overview_map = visualizer.create_overview_map(
    successful_fires,
    title="Post-Fire Debris Flow Hazard Assessment"
)

# 2. Individual fire summary sheets
for fire_name, project_info in successful_fires[:5]:  # First 5 as examples
    print(f"\n   Creating summary sheet for {fire_name}...")
    visualizer.create_fire_summary(
        fire_name=fire_name,
        project_dir=project_info['project_dir'],
        raster_dir=os.path.join(CONFIG['paths']['output_folder'], 'probability_rasters', fire_name)
    )

print("\n✅ All visualizations complete!")

## 9. Generate Final Report

In [ ]:
print("\n" + "="*60)
print("FINAL SUMMARY REPORT")
print("="*60)

# Calculate statistics
total_fires = len(fire_inventory)
successful_assessments = sum(1 for r in assessment_results.values() if r['success'])
total_area_km2 = sum(
    project_info.get('area_km2', 0) 
    for project_info in initialized_projects.values()
)

print(f"\n📊 PROCESSING STATISTICS:")
print(f"   Total fires processed: {total_fires}")
print(f"   Successful assessments: {successful_assessments}")
print(f"   Success rate: {successful_assessments/total_fires*100:.1f}%")
print(f"   Total area analyzed: {total_area_km2:.1f} km²")

# Processing time summary
if assessment_results:
    processing_times = [
        r['total_time'] 
        for r in assessment_results.values() 
        if r.get('success') and 'total_time' in r
    ]
    if processing_times:
        print(f"\n⏱️  PERFORMANCE METRICS:")
        print(f"   Average processing time: {sum(processing_times)/len(processing_times):.1f} seconds")
        print(f"   Total processing time: {sum(processing_times)/60:.1f} minutes")

# Output summary
print(f"\n📁 OUTPUT PRODUCTS:")
print(f"   Shapefiles: {successful_assessments * 3} files (basins, segments, outlets)")
print(f"   Probability rasters: {successful_assessments * 4} files (4 scenarios each)")
print(f"   Visualization maps: {successful_assessments + 1} files")

# Save final summary
summary = {
    'processing_date': time.strftime('%Y-%m-%d %H:%M:%S'),
    'configuration': CONFIG,
    'statistics': {
        'total_fires': total_fires,
        'successful': successful_assessments,
        'failed': total_fires - successful_assessments,
        'total_area_km2': total_area_km2
    },
    'outputs': {
        'assessment_results': 'assessment_results.json',
        'probability_rasters': f"{CONFIG['paths']['output_folder']}/probability_rasters/",
        'visualizations': f"{CONFIG['paths']['output_folder']}/visualizations/"
    }
}

with open('processing_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n✅ COMPLETE WORKFLOW FINISHED!")
print(f"📁 All results saved to: {CONFIG['paths']['output_folder']}")


## Next Steps
 
1. **Review Results**: Check the output folders for probability rasters and visualizations
2. **Quality Control**: Verify high-risk areas in GIS software
3. **Share Products**: Export maps and reports for stakeholders
4. **Archive Data**: Backup all processed data and results

### Troubleshooting
 
If any fires failed to process:
1. Check the error logs in `assessment_results.json`
2. Verify input data quality (DEM resolution, soil data completeness)
3. Adjust memory optimization settings if needed
4. Re-run failed fires individually

In [ ]:
# Display sample probability raster (if matplotlib available)
try:
    import matplotlib.pyplot as plt
    import rasterio
    from rasterio.plot import show
    
    # Find a sample raster
    sample_raster = None
    raster_folder = os.path.join(CONFIG['paths']['output_folder'], 'probability_rasters')
    
    if os.path.exists(raster_folder):
        for fire_folder in os.listdir(raster_folder):
            fire_path = os.path.join(raster_folder, fire_folder)
            if os.path.isdir(fire_path):
                for raster_file in os.listdir(fire_path):
                    if raster_file.endswith('_P_24mmh.tif'):
                        sample_raster = os.path.join(fire_path, raster_file)
                        break
                if sample_raster:
                    break
    
    if sample_raster and os.path.exists(sample_raster):
        print(f"\n📊 Sample probability raster: {os.path.basename(sample_raster)}")
        
        fig, ax = plt.subplots(figsize=(10, 8))
        with rasterio.open(sample_raster) as src:
            show(src, ax=ax, cmap='RdYlBu_r', title='Debris Flow Probability (24mm/hr scenario)')
        plt.colorbar(ax.images[0], label='Probability')
        plt.tight_layout()
        plt.show()
    else:
        print("\n📊 No sample rasters found for visualization")
        
except ImportError:
    print("\n📊 Matplotlib not available for visualization")
except Exception as e:
    print(f"\n📊 Could not display sample raster: {e}")